In [1]:
# example of using a pre-trained model as a classifier
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
# load an image from file
image = load_img('download.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# load the model
model = VGG16()
# predict the probability across all output classes
yhat = model.predict(image)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

40960/35363 [==================================] - 0s 0us/step
castle (34.03%)


In [5]:
# load an image from file
image = load_img('download2.png', target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# load the model
model = VGG16()
# predict the probability across all output classes
yhat = model.predict(image)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

valley (44.85%)


In [5]:
# ANOTHER CODE 
# --- Stage 1: Load MNIST dataset locally ---
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

# Load local dataset
data_path = r"mnist_dataset.npz"

with np.load(data_path) as data:
    X_train = data['X_train'][:10000]
    y_train = data['y_train'][:10000]
    X_test = data['X_test'][:2000]
    y_test = data['y_test'][:2000]

print("✅ Loaded MNIST dataset locally!")
print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

# --- Stage 2: Preprocess data ---
# MNIST images are 28x28 grayscale — add channel dim
x_train = np.expand_dims(X_train, -1)  # (28,28,1)
x_test = np.expand_dims(X_test, -1)

# Resize to 32x32 for VGG16 and convert to RGB (3 channels)
x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])
x_train = tf.image.grayscale_to_rgb(x_train)
x_test = tf.image.grayscale_to_rgb(x_test)

# Normalize pixel values to [0,1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

print("✅ Preprocessing complete.")
print("x_train:", x_train.shape, "y_train:", y_train.shape)
print("x_test :", x_test.shape, "y_test :", y_test.shape)

# --- Stage 3: Load pre-trained VGG16 (without top layers) ---
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze convolutional base layers
for layer in base_model.layers:
    layer.trainable = False

# --- Stage 4: Build the model ---
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')  # MNIST has 10 classes
])

# --- Stage 5: Compile the model ---
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# --- Stage 6: Train the model ---
history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=64,
    validation_data=(x_test, y_test),
    verbose=1
)

# --- Stage 7: Evaluate on test set ---
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")


✅ Loaded MNIST dataset locally!
Training data shape: (10000, 28, 28)
Test data shape: (2000, 28, 28)
✅ Preprocessing complete.
x_train: (10000, 32, 32, 3) y_train: (10000, 10)
x_test : (2000, 32, 32, 3) y_test : (2000, 10)
Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 111s 687ms/step - accuracy: 0.7569 - loss: 0.8457 - val_accuracy: 0.8465 - val_loss: 0.5126
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 110s 699ms/step - accuracy: 0.8968 - loss: 0.3603 - val_accuracy: 0.8935 - val_loss: 0.3445
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 109s 698ms/step - accuracy: 0.9237 - loss: 0.2639 - val_accuracy: 0.9165 - val_loss: 0.2786
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 112s 716ms/step - accuracy: 0.9354 - loss: 0.2161 - val_accuracy: 0.9305 - val_loss: 0.2376
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 108s 686ms/step - accuracy: 0.9430 - loss: 0.1900 - val_accuracy: 0.9345 - val_loss: 0.2145
✅ Test Accuracy: 0.9345
Test Loss: 0.2145


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

# Load local dataset
data_path = r"mnist_dataset.npz"
print(data.files)

['X_train', 'y_train', 'X_test', 'y_test']
